<a href="https://colab.research.google.com/github/aswanthoffl/DDD-Project/blob/main/Model%20Training/Drowsy_Yawn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd '/content/drive/MyDrive/EY_Dataset/dataset_new/yawn_dataset' 

/content/drive/MyDrive/EY_Dataset/dataset_new/yawn_dataset


In [ ]:
batchsize=32
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)
train_data= train_datagen.flow_from_directory(r'train',target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )
validation_data= train_datagen.flow_from_directory(r'train',target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'test',target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 987 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
Found 215 images belonging to 2 classes.


In [ ]:


# Load the pre-trained InceptionV3 model without the top layers
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

# Add new dense layers on top of the pre-trained model
model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
        layer.trainable = False


In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau,TensorBoard
checkpoint = ModelCheckpoint(r'models/model_yawn4.h5',monitor='val_loss',save_best_only=True,verbose=3)
earlystop = EarlyStopping(monitor = 'val_loss', patience=20, verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=10, verbose= 3 )
tensor_board= TensorBoard(r'Graph', histogram_freq=0, write_graph=True, write_images=True)
callbacks=[checkpoint,earlystop,learning_rate,tensor_board]

opt = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=300)

Epoch 1/300
30/30 [==============================] - ETA: 0s - loss: 0.5751 - accuracy: 0.6723
Epoch 1: val_loss did not improve from 0.61506
30/30 [==============================] - 29s 958ms/step - loss: 0.5751 - accuracy: 0.6723 - val_loss: 0.6388 - val_accuracy: 0.6250 - lr: 1.0000e-05
Epoch 2/300
30/30 [==============================] - ETA: 0s - loss: 0.5617 - accuracy: 0.6859
Epoch 2: val_loss did not improve from 0.61506
30/30 [==============================] - 22s 741ms/step - loss: 0.5617 - accuracy: 0.6859 - val_loss: 0.6553 - val_accuracy: 0.5759 - lr: 1.0000e-05
Epoch 3/300
30/30 [==============================] - ETA: 0s - loss: 0.5666 - accuracy: 0.6859
Epoch 3: val_loss did not improve from 0.61506
30/30 [==============================] - 24s 789ms/step - loss: 0.5666 - accuracy: 0.6859 - val_loss: 0.6288 - val_accuracy: 0.6562 - lr: 1.0000e-05
Epoch 4/300
30/30 [==============================] - ETA: 0s - loss: 0.5649 - accuracy: 0.6921
Epoch 4: val_loss did not improv

In [ ]:
test_accuracy = model.evaluate(test_data, steps=test_data.samples // batchsize) 
print("Test accuracy:", test_accuracy[1]) 

6/6 [==============================] - 41s 8s/step - loss: 0.6021 - accuracy: 0.6042
Test accuracy: 0.6041666865348816


In [ ]:
'''
adam =67.1   ----  65.1  
nadam = 66.6
adadelta=65.1
rmsprop = 63.5
SGD=62.5
adagrad=60.4

'''
'''
relu = "65.1" 64.05 
sigmoid="63.54"
tanh=60.4
LeakyRelu =55.7
'''
'''
binary_cross with relu =59.3

'''

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model 
model = load_model('/content/drive/MyDrive/EY_Dataset/dataset_new/yawn_dataset/models/model_yawn1.h5')


# Define the image size
IMAGE_SIZE = (80, 80)

# Load the image and preprocess it
mouth = cv2.imread('/content/drive/MyDrive/EY_Dataset/dataset_new/yawn_dataset/test/no_yawn/2090.jpg')
mouth= cv2.resize(mouth,(80, 80))
mouth= mouth/255
mouth=mouth.reshape(80, 80,3)
mouth= np.expand_dims(mouth,axis=0)
# preprocessing is done now model prediction
pred= model.predict(mouth)



if pred[0][1] > pred[0][0]:
    print('Yawn detected with score:', pred[0][1])
else:
    print('No Yawn detected with score:', pred[0][0])

1/1 [==============================] - 1s 1s/step
No Yawn detected with score: 0.85215604


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/EY_Dataset/dataset_new/yawn_dataset/models/model_yawn1.h5')

# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_data)

# Print the validation accuracy
print('Validation accuracy:', val_acc)

8/8 [==============================] - 6s 499ms/step - loss: 0.6341 - accuracy: 0.6098
Validation accuracy: 0.6097561120986938
